In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix

from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.layers import concatenate
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import models

In [3]:
path_data = "/content/drive/MyDrive/CTM_Hien-main/Data/"
path_result = "/content/drive/MyDrive/CTM_Hien-main/Result/"
path_model = "/content/drive/MyDrive/CTM_Hien-main/Model/"

In [ ]:
# Định nghĩa từ điển axit amin cấu tạo nên chuỗi protein
def twoTupleDic3(): # Từ điển 3-gram
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          for jj in AA_list_sort:
             AA_dict[i+j+jj] = numm
             numm += 1
    return AA_dict
def twoTupleDic2(): # Từ điển 2-gram
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        for j in AA_list_sort:
          AA_dict[i+j] = numm
          numm += 1
    return AA_dict

def twoTupleDic1(): # Từ điển 1-gram
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)

TIME_STEPS = 33
INPUT_SIZE = 300

In [5]:
# Tải dữ liệu train model
import pandas as pd
file_train = "train_data_3speaces_31.csv"
df_train=pd.read_csv(path_data + file_train, delimiter= ',')

texts_train =[] #PTMsequend kmer
for i in df_train['Sequence']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer train data input Word2vec
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())

MAX_SEQUENCE_LENGTH= len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
# perform one-hot encoding on the labels
lb = LabelBinarizer()
ytrain = lb.fit_transform(ytrain)
ytrain= to_categorical(ytrain)
ytrain.shape


(6746, 2)

In [6]:
# Tải dữ liệu test
file_test="train_data_3speaces_31.csv"
df_test =pd.read_csv(path_data +file_test,delimiter= ',')
text_test =[] #PTMsequend kmerpath_
for i in df_test['Sequence']:
  temp = ProSentence(i,k)
  text_test.append(temp)
df_test['k_mer'] =text_test

test_sequences = []
for each in text_test:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    test_sequences.append(each_index_list)

Xtest = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytest = np.array(df_test['Label'])
# perform one-hot encoding on the labels
ytest= np.array(ytest)
lb = LabelBinarizer()
ytest= lb.fit_transform(ytest)
ytest = to_categorical(ytest)
ytest.shape

(6746, 2)

ĐỊnh nghĩa mô hình chắt lọc tri thức

In [7]:
class Distiller(keras.Model):
    def __init__(self, student, teacher):
        super().__init__()
        self.teacher = teacher
        self.student = student

    def compile(
        self,
        optimizer,
        metrics,
        student_loss_fn,
        distillation_loss_fn,
        alpha=0.1,
        temperature=3,
    ):
        """ Configure the distiller.

        Args:
            optimizer: Keras optimizer for the student weights
            metrics: Keras metrics for evaluation
            student_loss_fn: Loss function of difference between student
                predictions and ground-truth
            distillation_loss_fn: Loss function of difference between soft
                student predictions and soft teacher predictions
            alpha: weight to student_loss_fn and 1-alpha to distillation_loss_fn
            temperature: Temperature for softening probability distributions.
                Larger temperature gives softer distributions.
        """
        super().compile(optimizer=optimizer, metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn
        self.alpha = alpha
        self.temperature = temperature

    def train_step(self, data):
        # Unpack data
        x, y = data

        # Forward pass of teacher
        teacher_predictions = self.teacher(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass of student
            student_predictions = self.student(x, training=True)

            # Compute losses
            student_loss = self.student_loss_fn(y, student_predictions)

            # Compute scaled distillation loss from https://arxiv.org/abs/1503.02531
            # The magnitudes of the gradients produced by the soft targets scale
            # as 1/T^2, multiply them by T^2 when using both hard and soft targets.
            distillation_loss = (
                self.distillation_loss_fn(
                    tf.nn.softmax(teacher_predictions / self.temperature, axis=1),
                    tf.nn.softmax(student_predictions / self.temperature, axis=1),
                )
                * self.temperature**2
            )

            loss = self.alpha * student_loss + (1 - self.alpha) * distillation_loss

        # Compute gradients
        trainable_vars = self.student.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Update the metrics configured in `compile()`.
        self.compiled_metrics.update_state(y, student_predictions)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update(
            {"student_loss": student_loss, "distillation_loss": distillation_loss}
        )
        return results

    def test_step(self, data):
        # Unpack the data
        x, y = data

        # Compute predictions
        y_prediction = self.student(x, training=False)

        # Calculate the loss
        student_loss = self.student_loss_fn(y, y_prediction)

        # Update the metrics.
        self.compiled_metrics.update_state(y, y_prediction)

        # Return a dict of performance
        results = {m.name: m.result() for m in self.metrics}
        results.update({"student_loss": student_loss})
        return results
teacher = keras.Sequential(
    [
        layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True),
        #layers.dropout(0.2),
        layers.LSTM(300, dropout=0.2, recurrent_dropout=0.2),
        layers.Dropout(0.2),
        #layers.LSTM(300, dropout=0.2, recurrent_dropout=0.2),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.2),
        layers.Dense(2),
        layers.Activation('softmax'),

    ],
    name="teacher",
)

teacher.compile(
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[
        "categorical_accuracy",
        "AUC",

    ],
    optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
)
student = keras.Sequential(
    [
        layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True),
        #layers.dropout(0.2),
        layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
        layers.Dropout(0.2),
        #layers.LSTM(300, dropout=0.2, recurrent_dropout=0.2),
        layers.Flatten(),
        layers.Dense(128),
        layers.Dropout(0.2),
        layers.Dense(2),
        layers.Activation('softmax'),
    ],
    name="student",
)
student_scratch = keras.models.clone_model(student)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [8]:
checkpoint = EarlyStopping(monitor='val_loss',
            min_delta=0,
            patience=3,
            verbose=1, mode='auto')
result_test =path_result +"Result_KD.txt"


Knowledge Distillation base on LSTM

In [ ]:
# Huấn luyện mô hình teacher trên bộ dữ liệu Xtrain, ytrain
teacher.fit(Xtrain,ytrain, batch_size=16, epochs=50)#,validation_data =(Xtest_teacher, ytest_teacher), shuffle = True,callbacks=[checkpoint],verbose=1)
teacher.save(path_model +'model_teacher.h5')
ypred = teacher.predict(Xtest)
ypred =np.argmax(ypred,axis =1)

ytest_true =  np.argmax(ytest,axis =1)

result22 = confusion_matrix(ytest_true,ypred)
print(result22)

Epoch 1/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 64s 141ms/step - AUC: 0.6579 - categorical_accuracy: 0.6101 - loss: 0.6493
Epoch 2/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 79s 134ms/step - AUC: 0.7817 - categorical_accuracy: 0.7154 - loss: 0.5684
Epoch 3/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 84s 140ms/step - AUC: 0.7834 - categorical_accuracy: 0.7241 - loss: 0.5683
Epoch 4/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 81s 138ms/step - AUC: 0.8012 - categorical_accuracy: 0.7416 - loss: 0.5485
Epoch 5/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 82s 139ms/step - AUC: 0.7938 - categorical_accuracy: 0.7315 - loss: 0.5554
Epoch 6/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 56s 132ms/step - AUC: 0.8171 - categorical_accuracy: 0.7492 - loss: 0.5296
Epoch 7/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 85s 138ms/step - AUC: 0.8167 - categorical_accuracy: 0.7472 - loss: 0.5270
Epoch 8/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 84s 144ms/step - AUC: 0.8286 - categorical_accuracy: 0.7665 - loss: 0.5142
Epoch 9/50
422/422 ━━━━━━━━━━━━━━━━━━━━ 79s 138ms/step - AUC: 0.8294 - categoric

In [ ]:
# Huấn luyện model student học chuyển giao từ teacher
teacher_model = models.load_model(path_model + 'model_teacher.h5') # Tải model teacher đã được huấn luyện
distiller = Distiller(student=student, teacher=teacher_model)
distiller.compile(
      student_loss_fn= keras.losses.CategoricalCrossentropy(from_logits=True),
      #student_loss_fn= keras.losses.BinaryCrossentropy(from_logits=True),
      optimizer=keras.optimizers.AdamW(learning_rate = 0.0001),
      metrics=[
              "categorical_accuracy",
              "AUC",
              ], # , f1
      distillation_loss_fn=keras.losses.KLDivergence(),
      alpha=0.1,
      temperature=10
      )

history_KD = distiller.fit(Xtrain,ytrain, batch_size=16,epochs=50)#, validation_data =(Xtest_student, ytest_student), callbacks=[checkpoint],verbose=1)
result2 = distiller.evaluate(Xtest, ytest)#, validation_data=(X_test,Y_test), shuffle = True,callbacks=[checkpoint],verbose=1)
f = open(result_test, 'a+', encoding='UTF-8')
f.write("\n Independent model KD_BiLSTM  \n ")
s = str(result2)
f.write(s)
f.close()

ypred = student.predict(Xtest)
ypred =np.argmax(ypred,axis =1)

ytest_true =  np.argmax(ytest,axis =1)

result22 = confusion_matrix(ytest_true,ypred)
print(result22)
f = open(result_test, 'a+', encoding='UTF-8')
f.write("\n confusion_matrix KD_BIlstm: Student \n ")
s = str(result22)
f.write(s)
f.close()
# save model student
student.save(path_model +"KD_LSTM.h5")

**Tải model student dự đoán cho dữ liệu test**

In [ ]:
from keras import models
model_KD = models.load_model(path_model +"KD_LSTM.h5")
y_prob_KD = model_KD.predict(Xtest)
pd.DataFrame(y_prob_KD[:,1], columns=['KD_pred']).to_csv(path_result +'KD_prediction.csv')

**Huấn luyện độc lập mô hình Teacher, student để so sánh với mô hình chắt lọc tri thức**

In [ ]:
# Model Student test independent base on Student dataset
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Flatten, Dense
# Define your student model
model_student = Sequential()
model_student.add(Embedding(vocab_size + 1, 300, input_length=MAX_SEQUENCE_LENGTH_student, trainable=True))
model_student.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_student.add(Dropout(0.2))
model_student.add(Flatten())
model_student.add(Dense(128, activation='relu'))
model_student.add(Dropout(0.2))
model_student.add(Dense(2, activation='softmax'))

metrics=[
          "accuracy",
          "AUC",
          ]
model_student.compile(optimizer=keras.optimizers.AdamW(learning_rate = 0.0001), loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=metrics)

# Train your model
history_model_student = model_student.fit(Xtrain_student, ytrain_student, batch_size=16, epochs=50, validation_data=(Xtest_student, ytest_student),callbacks=[checkpoint],verbose=1)

# Evaluate your model
result2 = model_student.evaluate(Xtest_student, ytest_student)

# Save evaluation results to a file
with open(result_test, 'a+', encoding='UTF-8') as f:
    f.write("\n Model Student: Test independent \n ")
    f.write(str(result2))

# Make predictions
ypred = model_student.predict(Xtest_student)
ypred = np.argmax(ypred, axis=1)

# Calculate confusion matrix
result22 = confusion_matrix(np.argmax(ytest_student, axis=1), ypred)
print(result22)
# Save confusion matrix to a file
with open(result_test, 'a+', encoding='UTF-8') as f:
    f.write("\n model Student: Confusion matrix \n ")
    f.write(str(result22))
model_student.save(path +"model_student.h5")

In [ ]:
# Model Teacher test independent base on Student dataset
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Flatten, Dense

# Define your teacher model
model_teacher = Sequential()
model_teacher.add(Embedding(vocab_size + 1, 300, input_length=MAX_SEQUENCE_LENGTH_student, trainable=True))
model_teacher.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
model_teacher.add(Dropout(0.2))
model_teacher.add(Flatten())
model_teacher.add(Dense(128, activation='relu'))
model_teacher.add(Dropout(0.2))
model_teacher.add(Dense(2, activation='softmax'))

metrics=[
          "accuracy",
          "AUC",
          ]
model_teacher.compile(optimizer=keras.optimizers.AdamW(learning_rate = 0.0001), loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=metrics)

# Train your model
history_model_teacher = model_teacher.fit(Xtrain_student, ytrain_student, batch_size=16, epochs=50, validation_data=(Xtest_student, ytest_student),callbacks=[checkpoint],verbose=1)

# Evaluate your model
result2 = model_teacher.evaluate(Xtest_student, ytest_student)

# Save evaluation results to a file
with open(result_test, 'a+', encoding='UTF-8') as f:
    f.write("\n Model teacher: Independent test \n ")
    f.write(str(result2))

# Make predictions
ypred = model_teacher.predict(Xtest_student)
ypred = np.argmax(ypred, axis=1)

# Calculate confusion matrix
result22 = confusion_matrix(np.argmax(ytest_student, axis=1), ypred)
print(result22)
# Save confusion matrix to a file
with open(result_test, 'a+', encoding='UTF-8') as f:
    f.write("\n Model teacher: Confusion matric \n ")
    f.write(str(result22))
model_teacher.save(path +"model_teacher.h5")

In [ ]:
#cross-validation teacher

from sklearn.model_selection import KFold
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dropout, Flatten, Dense
from sklearn.metrics import confusion_matrix


kf = KFold(n_splits=num_folds, shuffle=True)

X_student = Xtrain_student
Y_student = ytrain_student

for train_index_student, test_index_student in kf.split(X_student,Y_student):
    X_train_student, X_test_student =X_student[train_index_student], X_student[test_index_student]
    Y_train_student, Y_test_student = Y_student[train_index_student], Y_student[test_index_student]



    model_teacher = Sequential()
    model_teacher.add(Embedding(vocab_size + 1, 300, input_length=MAX_SEQUENCE_LENGTH_student, trainable=True))
    model_teacher.add(LSTM(300, dropout=0.2, recurrent_dropout=0.2))
    model_teacher.add(Dropout(0.2))
    model_teacher.add(Flatten())
    model_teacher.add(Dense(128, activation='relu'))
    model_teacher.add(Dropout(0.2))
    model_teacher.add(Dense(2, activation='softmax'))
    model_teacher.compile(optimizer=keras.optimizers.AdamW(learning_rate = 0.0001), loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy','AUC'])
    # Train your model
    history_model_teacher = model_teacher.fit(X_train_student, Y_train_student, batch_size=16, epochs=50, validation_data=(X_test_student, Y_test_student),callbacks=[checkpoint],verbose=1)
    # Evaluate your model
    result2 = model_teacher.evaluate(X_test_student, Y_test_student)
    # Save evaluation results to a file
    with open(result_test, 'a+', encoding='UTF-8') as f:
        f.write("\n Cross validation Model teacher \n ")
        f.write(str(result2))
    # Make predictions
    ypred = model_teacher.predict(X_test_student)
    ypred = np.argmax(ypred, axis=1)
    # Calculate confusion matrix
    result22 = confusion_matrix(np.argmax(Y_test_student, axis=1), ypred)
    print(result22)
    # Save confusion matrix to a file
    with open(result_test, 'a+', encoding='UTF-8') as f:
        f.write("\n model teacher \n ")
        f.write(str(result22))

In [ ]:
# kiểm thử chéo model sinh viên
from sklearn.model_selection import KFold
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True)

X_student = Xtrain_student
Y_student = ytrain_student

for train_index_student, test_index_student in kf.split(X_student,Y_student):
    X_train_student, X_test_student =X_student[train_index_student], X_student[test_index_student]
    Y_train_student, Y_test_student = Y_student[train_index_student], Y_student[test_index_student]
    model_student = Sequential()
    model_student.add(Embedding(vocab_size + 1, 300, input_length=MAX_SEQUENCE_LENGTH_student, trainable=True))
    model_student.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model_student.add(Dropout(0.2))
    model_student.add(Flatten())
    model_student.add(Dense(128, activation='relu'))
    model_student.add(Dropout(0.2))
    model_student.add(Dense(2, activation='softmax'))
    model_student.compile(optimizer=keras.optimizers.AdamW(learning_rate = 0.0001), loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy','AUC'])

    # Train your model
    history_model_student = model_student.fit(X_train_student, Y_train_student, batch_size=16, epochs=50, validation_data=(X_test_student, Y_test_student),callbacks=[checkpoint],verbose=1)

    # Evaluate your model
    result2 = model_student.evaluate(X_test_student, Y_test_student)

    # Save evaluation results to a file
    with open(result_test, 'a+', encoding='UTF-8') as f:
        f.write("\n Cross validation Model Student \n ")
        f.write(str(result2))

    # Make predictions
    ypred = model_student.predict(X_test_student)
    ypred = np.argmax(ypred, axis=1)

    # Calculate confusion matrix
    result22 = confusion_matrix(np.argmax(Y_test_student, axis=1), ypred)
    print(result22)
    # Save confusion matrix to a file
    with open(result_test, 'a+', encoding='UTF-8') as f:
        f.write("\n model Student \n ")
        f.write(str(result22))


Models predict and draw ROC

In [ ]:
from keras import models
model_teacher = models.load_model(path +"model_teacher.h5")


#y_prob_teacher = np.array(list(map(predict_prob, model_teacher.predict(Xtest_student))))
y_prob_teacher = model_teacher.predict(Xtest_student)
pd.DataFrame(y_prob_teacher[:,1], columns=['predictions']).to_csv(path +'teacher_prediction.csv')

In [ ]:
from keras import models
model_student = models.load_model(path +"model_student.h5")


#y_prob_st = np.array(list(map(predict_prob, model_student.predict(Xtest_student))))
y_prob_st =model_student.predict(Xtest_student)
pd.DataFrame(y_prob_st[:,1], columns=['student_pred']).to_csv(path +'student_prediction.csv')

In [ ]:
from keras import models
model_KD = models.load_model(path +"KD_LSTM.h5")
y_prob_KD = model_KD.predict(Xtest_student)
pd.DataFrame(y_prob_KD[:,1], columns=['KD_pred']).to_csv(path +'KD_prediction.csv')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import numpy as np
from keras import models
from sklearn import metrics
# Generate some sample data (replace this with your actual data)
# Assuming y_true and y_pred are your true labels and predicted probabilities respectively


y_true=ytest_student[:,1]

# Calculate ROC curve and AUC for Teacher

#fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=2)
fpr_teacher, tpr_teacher, thresholds_teacher  = metrics.roc_curve(y_true, y_prob_teacher[:,1])
roc_auc_teacher = auc(fpr_teacher, tpr_teacher)

# Calculate ROC curve and AUC for Student
fpr_student, tpr_student, thresholds_student = metrics.roc_curve(y_true, y_prob_st[:,1])
roc_auc_student = auc(fpr_student, tpr_student)
# Calculate ROC curve and AUC for Knowlege distillation
fpr_KD, tpr_KD, thresholds_KD = metrics.roc_curve(y_true, y_prob_KD[:,1])
roc_auc_KD = auc(fpr_KD, tpr_KD)
# Plot ROC curves
plt.figure(figsize=(8, 6))
plt.plot(fpr_teacher, tpr_teacher, color='red', lw=2, label='Teacher model, AUC = {:.4f}'.format(0.903))
plt.plot(fpr_student, tpr_student,  color='red', lw=2, label='Student model, AUC = {:.4f}'.format(roc_auc_student))
plt.plot(fpr_KD, tpr_KD, color='green',lw=2, label='Knowledge Distillation, AUC = {:.4f}'.format(roc_auc_KD))
plt.plot([0, 1], [0, 1], color='black', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

In [ ]:
distiller.summary()